## About this demo
This demo describes step-by-step how to use BMF to develop a transcoding program, including video transcoding, audio transcoding, and image transcoding. In it, you can familiarize yourself with how to use BMF and how to use FFmpeg-compatible options to achieve the capabilities you need.

## 1. Environmental preparation

### 1.1 FFmpeg
FFmpeg 4.x or 5.x is needed by BMF when transcoding, check versions via apt:

In [12]:
! apt show ffmpeg libavcodec-dev libavdevice-dev libavfilter-dev libavformat-dev libavresample-dev libavutil-dev libpostproc-dev libswresample-dev libswscale-dev | grep "^Package:\|^Version:"



Package: ffmpeg
Version: 7:4.2.7-0ubuntu0.1
Package: libavcodec-dev
Version: 7:4.2.7-0ubuntu0.1
Package: libavdevice-dev
Version: 7:4.2.7-0ubuntu0.1
Package: libavfilter-dev
Version: 7:4.2.7-0ubuntu0.1
Package: libavformat-dev
Version: 7:4.2.7-0ubuntu0.1
Package: libavresample-dev
Version: 7:4.2.7-0ubuntu0.1
Package: libavutil-dev
Version: 7:4.2.7-0ubuntu0.1
Package: libpostproc-dev
Version: 7:4.2.7-0ubuntu0.1
Package: libswresample-dev
Version: 7:4.2.7-0ubuntu0.1
Package: libswscale-dev
Version: 7:4.2.7-0ubuntu0.1


If the version meets the requirements, install ffmpeg via apt:

In [13]:
! apt install -y ffmpeg libavcodec-dev libavdevice-dev libavfilter-dev libavformat-dev libavresample-dev libavutil-dev libpostproc-dev libswresample-dev libswscale-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
libavcodec-dev is already the newest version (7:4.2.7-0ubuntu0.1).
libavdevice-dev is already the newest version (7:4.2.7-0ubuntu0.1).
libavfilter-dev is already the newest version (7:4.2.7-0ubuntu0.1).
libavformat-dev is already the newest version (7:4.2.7-0ubuntu0.1).
libavresample-dev is already the newest version (7:4.2.7-0ubuntu0.1).
libavutil-dev is already the newest version (7:4.2.7-0ubuntu0.1).
libpostproc-dev is already the newest version (7:4.2.7-0ubuntu0.1).
libswresample-dev is already the newest version (7:4.2.7-0ubuntu0.1).
libswscale-dev is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


Otherwise, you need to compile ffmpeg from source, you can use the script we provided(only linux and macos now):

In [ ]:
! git clone https://github.com/BabitMF/bmf bmf
! cd bmf
! ./scripts/build_ffmpeg.sh x264 x265

### 1.2 BMF
BMF can be installed in many ways, we use pip here:

In [14]:
! pip3 install BabitMF

### 1.3 wurlitzer(optional)
This package is installed to show the BMF C++ logs in the colab console, otherwise only python logs are printed. This step is not necessary if you're not in a Colab or iPython notebook environment.

In [15]:
!pip install wurlitzer
%load_ext wurlitzer

The wurlitzer extension is already loaded. To reload it, use:
  %reload_ext wurlitzer


## 2. Transcode demo

Download the video file we will be using first:

In [16]:
!gdown --fuzzy https://drive.google.com/file/d/1l8bDSrWn6643aDhyaocVStXdoUbVC3o2/view?usp=sharing -O big_bunny_10s_30fps.mp4

Downloading...
From: https://drive.google.com/uc?id=1l8bDSrWn6643aDhyaocVStXdoUbVC3o2
To: /content/big_bunny_10s_30fps.mp4
100% 2.56M/2.56M [00:00<00:00, 141MB/s]


In [17]:
! ffprobe big_bunny_10s_30fps.mp4

ffprobe version 4.2.7-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --

### 2.1 video transcoding

#### 2.1.1 remuxing demo
This demo shows how to demux a video in mp4 format and remuxing it into hls slices without involving audio and video decoding and encoding:


In [18]:
import bmf

input_video_path = "./big_bunny_10s_30fps.mp4"
output_path = "./remux_output.m3u8"

# create graph
graph = bmf.graph()

# decode
video = graph.decode({
    "input_path": input_video_path,
    "video_codec": "copy",
    "audio_codec": "copy"
})

(
    bmf.encode(
        video['video'],
        video['audio'],
        {
            "output_path": output_path,
            "format": "hls",
            "mux_params": {
                "hls_list_size": "0",
                "hls_time": "2",
                "hls_segment_filename": "./file%03d.ts"
            }
        }
    ).run()
)

{
    "input_streams": [],
    "output_streams": [],
    "nodes": [
        {
            "module_info": {
                "name": "c_ffmpeg_decoder",
                "type": "",
                "path": "",
                "entry": ""
            },
            "meta_info": {
                "premodule_id": -1,
                "callback_binding": []
            },
            "option": {
                "input_path": "./big_bunny_10s_30fps.mp4",
                "video_codec": "copy",
                "audio_codec": "copy"
            },
            "input_streams": [],
            "output_streams": [
                {
                    "identifier": "video:c_ffmpeg_decoder_14_1",
                    "stream_alias": ""
                },
                {
                    "identifier": "audio:c_ffmpeg_decoder_14_2",
                    "stream_alias": ""
                }
            ],
            "input_manager": "immediate",
            "scheduler": 0,
            "alias": "",
  

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './big_bunny_10s_30fps.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.76.100
  Duration: 00:00:10.00, start: 0.000000, bitrate: 2044 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 1920x1080 [SAR 1:1 DAR 16:9], 1904 kb/s, 30 fps, 30 tbr, 15360 tbn, 60 tbc (default)

[2023-07-11 08:03:20.395] [info] c++ module constructed
[2023-07-11 08:03:20.396] [info] Constructing c++ module
[2023-07-11 08:03:20.396] [info] c++ module constructed
[2023-07-11 08:03:20.396] [info] BMF Version: 0.0.5
[2023-07-11 08:03:20.396] [info] BMF Commit: 98e17ef
[2023-07-11 08:03:20.396] [info] start init graph
[2023-07-11 08:03:20.396] [info] scheduler count2
debug queue size, node 14, queue size: 5
[2023-07-11 08:03:20.396] [info] node:c_ffmpeg_decoder 14 scheduler 0
debug queue size, node 15, queue size: 5
[2023-07-11 08:03:20.396] [info] node:c_ffmpeg_encoder 15 scheduler 1



    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 129 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
[hls @ 0x7f36001c0480] Opening './file000.ts' for writing
Output #0, hls, to './remux_output.m3u8':
  Metadata:
    encoder         : Lavf58.29.100
    Stream #0:0: Video: h264 (High) (avc1 / 0x31637661), yuv420p, 1920x1080 [SAR 1:1 DAR 16:9], q=2-31, 1904 kb/s, 30 fps, 30 tbr, 90k tbn, 15360 tbc
    Stream #0:1: Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 129 kb/s
[hls @ 0x7f36001c0480] Opening './remux_output.m3u8.tmp' for writing
[hls @ 0x7f36001c0480] Opening './file001.ts' for writing


[2023-07-11 08:03:20.485] [info] node id:14 decode flushing
[2023-07-11 08:03:20.485] [info] node id:14 Process node end
[2023-07-11 08:03:20.486] [info] node id:14 close node
[2023-07-11 08:03:20.486] [info] node 14 close report, closed count: 1
[2023-07-11 08:03:20.486] [info] node id:15 eof received
[2023-07-11 08:03:20.486] [info] node id:15 eof processed, remove node from scheduler
[2023-07-11 08:03:20.486] [info] node id:15 eof received
[2023-07-11 08:03:20.486] [info] node id:15 eof processed, remove node from scheduler
[2023-07-11 08:03:20.486] [info] node id:15 process eof, add node to scheduler
[2023-07-11 08:03:20.487] [info] node id:15 Process node end
[2023-07-11 08:03:20.487] [info] node id:15 close node
[2023-07-11 08:03:20.487] [info] node 15 close report, closed count: 2
[2023-07-11 08:03:20.487] [info] schedule queue 0 start to join thread


[hls @ 0x7f36001c0480] Opening './remux_output.m3u8.tmp' for writing


[2023-07-11 08:03:20.487] [info] schedule queue 0 thread quit
[2023-07-11 08:03:20.487] [info] schedule queue 0 closed
[2023-07-11 08:03:20.487] [info] schedule queue 1 start to join thread
[2023-07-11 08:03:20.487] [info] schedule queue 1 thread quit
[2023-07-11 08:03:20.488] [info] schedule queue 1 closed
[2023-07-11 08:03:20.488] [info] all scheduling threads were joint


In [19]:
! ffprobe remux_output.m3u8
! rm -rf remux_output.m3u8 file*.ts

ffprobe version 4.2.7-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --

#### 2.1.2 decoding and encoding demo
This demo shows how to decode a H.264 video, drop the audio, scale the video to 720x576p resolution, and encode it with x265 encoder. You can even do it in one line of code:

In [20]:
import bmf

input_video_path = "./big_bunny_10s_30fps.mp4"
output_path = "./decode_scale_encode_output.mp4"
(
    bmf.graph()
        .decode({'input_path': input_video_path})['video']
        .scale(720, 576)
        .encode(None, {
            "output_path": output_path,
            "video_params": {
                "codec": "libx265"
            }
        }).run()
)

{
    "input_streams": [],
    "output_streams": [],
    "nodes": [
        {
            "module_info": {
                "name": "c_ffmpeg_decoder",
                "type": "",
                "path": "",
                "entry": ""
            },
            "meta_info": {
                "premodule_id": -1,
                "callback_binding": []
            },
            "option": {
                "input_path": "./big_bunny_10s_30fps.mp4"
            },
            "input_streams": [],
            "output_streams": [
                {
                    "identifier": "video:c_ffmpeg_decoder_16_1",
                    "stream_alias": ""
                }
            ],
            "input_manager": "immediate",
            "scheduler": 0,
            "alias": "",
            "id": 16
        },
        {
            "module_info": {
                "name": "c_ffmpeg_filter",
                "type": "",
                "path": "",
                "entry": ""
            },
        

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './big_bunny_10s_30fps.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.76.100
  Duration: 00:00:10.00, start: 0.000000, bitrate: 2044 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 1920x1080 [SAR 1:1 DAR 16:9], 1904 kb/s, 30 fps, 30 tbr, 15360 tbn, 60 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 129 kb/s (default)
    Metadata:
      handler_name    : SoundHandler


[2023-07-11 08:04:23.686] [info] c++ module constructed
[2023-07-11 08:04:23.686] [info] Constructing c++ module
[2023-07-11 08:04:23.686] [info] c++ module constructed
[2023-07-11 08:04:23.686] [info] Constructing c++ module
[2023-07-11 08:04:23.686] [info] c++ module constructed
[2023-07-11 08:04:23.694] [info] BMF Version: 0.0.5
[2023-07-11 08:04:23.694] [info] BMF Commit: 98e17ef
[2023-07-11 08:04:23.694] [info] start init graph
[2023-07-11 08:04:23.699] [info] scheduler count2
debug queue size, node 16, queue size: 5
[2023-07-11 08:04:23.699] [info] node:c_ffmpeg_decoder 16 scheduler 0
debug queue size, node 18, queue size: 5
[2023-07-11 08:04:23.699] [info] node:c_ffmpeg_encoder 18 scheduler 1
debug queue size, node 17, queue size: 5
[2023-07-11 08:04:23.699] [info] Constructing c++ module
[2023-07-11 08:04:23.699] [info] c++ module constructed
[2023-07-11 08:04:23.699] [info] node:c_ffmpeg_filter 17 scheduler 0


x265 [info]: HEVC encoder version 3.2.1+1-b5c86a64bbbe
x265 [info]: build info [Linux][GCC 9.3.0][64 bit] 8bit+10bit+12bit
x265 [info]: using cpu capabilities: MMX2 SSE2Fast LZCNT SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
x265 [info]: Main profile, Level-3 (Main tier)
x265 [info]: Thread pool created using 2 threads
x265 [info]: Slices                              : 1
x265 [info]: frame threads / pool features       : 1 / wpp(9 rows)
x265 [warning]: Source height < 720p; disabling lookahead-slices
set_mempolicy: Operation not permitted
set_mempolicy: Operation not permitted
set_mempolicy: Operation not permitted
set_mempolicy: Operation not permitted
set_mempolicy: Operation not permitted
set_mempolicy: Operation not permitted
x265 [info]: Coding QT: max CU size, min CU size : 64 / 8
x265 [info]: Residual QT: max TU size, max depth : 32 / 1 inter / 1 intra
x265 [info]: ME / range / subpel / merge         : hex / 57 / 2 / 3
x265 [info]: Keyframe min / max / scenecut / bias: 25 / 250 / 40 / 5.00
x

[2023-07-11 08:04:33.837] [info] node id:16 decode flushing
[2023-07-11 08:04:33.837] [info] node id:16 Process node end
[2023-07-11 08:04:33.842] [info] node id:16 close node
[2023-07-11 08:04:33.842] [info] node 16 close report, closed count: 1
[2023-07-11 08:04:33.842] [info] node id:17 eof received
[2023-07-11 08:04:34.270] [info] node id:17 eof processed, remove node from scheduler
[2023-07-11 08:04:34.271] [info] node id:17 process eof, add node to scheduler
[2023-07-11 08:04:34.304] [info] node id:17 Process node end
[2023-07-11 08:04:34.308] [info] node id:17 close node
[2023-07-11 08:04:34.311] [info] node 17 close report, closed count: 2
[2023-07-11 08:04:34.314] [info] node id:18 eof received
[2023-07-11 08:04:34.314] [info] node id:18 eof processed, remove node from scheduler
[2023-07-11 08:04:35.205] [info] node id:18 process eof, add node to scheduler
[2023-07-11 08:04:35.920] [info] node id:18 Process node end


x265 [info]: frame I:      3, Avg QP:31.10  kb/s: 163.76  
x265 [info]: frame P:     74, Avg QP:31.45  kb/s: 570.11  
x265 [info]: frame B:    223, Avg QP:35.46  kb/s: 20.94   
x265 [info]: Weighted P-Frames: Y:17.6% UV:17.6%
x265 [info]: consecutive B-frames: 9.1% 3.9% 6.5% 49.4% 31.2% 

encoded 300 frames in 12.13s (24.73 fps), 157.83 kb/s, Avg QP:34.42


[2023-07-11 08:04:35.921] [info] node id:18 close node
[2023-07-11 08:04:35.921] [info] node 18 close report, closed count: 3
[2023-07-11 08:04:35.921] [info] schedule queue 0 start to join thread
[2023-07-11 08:04:35.921] [info] schedule queue 0 thread quit
[2023-07-11 08:04:35.922] [info] schedule queue 0 closed
[2023-07-11 08:04:35.922] [info] schedule queue 1 start to join thread
[2023-07-11 08:04:35.922] [info] schedule queue 1 thread quit
[2023-07-11 08:04:35.931] [info] schedule queue 1 closed
[2023-07-11 08:04:35.931] [info] all scheduling threads were joint


In [21]:
! ffprobe decode_scale_encode_output.mp4
! rm -rf decode_scale_encode_output.mp4

ffprobe version 4.2.7-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --

#### 2.1.3 multi stream demo
This demo shows how to decode the original video, use the ffmpeg filter capability, and encode multi videos with different options:

In [22]:
import bmf

input_video_path = "./big_bunny_10s_30fps.mp4"
output_path0 = "./decode_encode_multi_output0.mp4"
output_path1 = "./decode_encode_multi_output1.mp4"

streams = bmf.graph().decode({'input_path': input_video_path})
split_streams = streams['video'].split()
bmf.encode(split_streams[0], streams['audio'], {
    "output_path": output_path0,
    "video_params": {
        "codec": "libx264",
        "x264-params": "ssim=1:psnr=1"
    }
})
(
    bmf.encode(split_streams[1], streams['audio'], {
            "output_path": output_path1,
            "video_params": {
                "codec": "libx265",
                "preset": "fast",
                "crf": "23"
            }
        }).run()
)

{
    "input_streams": [],
    "output_streams": [],
    "nodes": [
        {
            "module_info": {
                "name": "c_ffmpeg_decoder",
                "type": "",
                "path": "",
                "entry": ""
            },
            "meta_info": {
                "premodule_id": -1,
                "callback_binding": []
            },
            "option": {
                "input_path": "./big_bunny_10s_30fps.mp4"
            },
            "input_streams": [],
            "output_streams": [
                {
                    "identifier": "video:c_ffmpeg_decoder_19_1",
                    "stream_alias": ""
                },
                {
                    "identifier": "audio:c_ffmpeg_decoder_19_2",
                    "stream_alias": ""
                }
            ],
            "input_manager": "immediate",
            "scheduler": 0,
            "alias": "",
            "id": 19
        },
        {
            "module_info": {
         

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './big_bunny_10s_30fps.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.76.100
  Duration: 00:00:10.00, start: 0.000000, bitrate: 2044 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 1920x1080 [SAR 1:1 DAR 16:9], 1904 kb/s, 30 fps, 30 tbr, 15360 tbn, 60 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 129 kb/s (default)
    Metadata:
      handler_name    : SoundHandler


[2023-07-11 08:05:41.660] [info] c++ module constructed
[2023-07-11 08:05:41.660] [info] Constructing c++ module
[2023-07-11 08:05:41.660] [info] c++ module constructed
[2023-07-11 08:05:41.660] [info] Constructing c++ module
[2023-07-11 08:05:41.660] [info] c++ module constructed
[2023-07-11 08:05:41.660] [info] Constructing c++ module
[2023-07-11 08:05:41.660] [info] c++ module constructed
[2023-07-11 08:05:41.660] [info] BMF Version: 0.0.5
[2023-07-11 08:05:41.660] [info] BMF Commit: 98e17ef
[2023-07-11 08:05:41.660] [info] start init graph
[2023-07-11 08:05:41.660] [info] scheduler count2
debug queue size, node 19, queue size: 5
[2023-07-11 08:05:41.660] [info] node:c_ffmpeg_decoder 19 scheduler 0
debug queue size, node 21, queue size: 5
[2023-07-11 08:05:41.661] [info] node:c_ffmpeg_encoder 21 scheduler 1
debug queue size, node 22, queue size: 5
[2023-07-11 08:05:41.661] [info] node:c_ffmpeg_encoder 22 scheduler 1
debug queue size, node 20, queue size: 5
[2023-07-11 08:05:41.661] 

[libx264 @ 0x7f3628261f00] --psnr used with psy on: results will be invalid!
[libx264 @ 0x7f3628261f00] --tune psnr should be used if attempting to benchmark psnr!
[libx264 @ 0x7f3628261f00] using SAR=1/1
[libx264 @ 0x7f3628261f00] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x7f3628261f00] profile High, level 4.0
[libx264 @ 0x7f3628261f00] 264 - core 155 r2917 0a84d98 - H.264/MPEG-4 AVC codec - Copyleft 2003-2018 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=3 lookahead_threads=1 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0

[2023-07-11 08:05:43.626] [info] schedule queue 0 start to join thread
[2023-07-11 08:05:43.626] [info] schedule queue 0 closed
[2023-07-11 08:05:43.626] [info] schedule queue 1 start to join thread
[2023-07-11 08:05:43.626] [info] schedule queue 1 closed
[2023-07-11 08:05:43.626] [info] all scheduling threads were joint
[2023-07-11 08:05:43.626] [info] node id:16 video frame decoded:300
[2023-07-11 08:05:43.626] [info] node id:16 audio frame decoded:0, sample decoded:0
[2023-07-11 08:06:36.800] [info] node id:19 decode flushing
[2023-07-11 08:06:36.800] [info] node id:19 Process node end
[2023-07-11 08:06:36.806] [info] node id:19 close node
[2023-07-11 08:06:36.806] [info] node 19 close report, closed count: 1
[2023-07-11 08:06:36.806] [info] node id:20 eof received
[2023-07-11 08:06:36.806] [info] node id:20 eof processed, remove node from scheduler
[2023-07-11 08:06:36.807] [info] node id:21 eof received
[2023-07-11 08:06:36.807] [info] node id:22 eof received
[2023-07-11 08:06:36.

...

[2023-07-11 08:06:47.248] [info] node id:21 close node
[2023-07-11 08:06:47.248] [info] node 21 close report, closed count: 3
[2023-07-11 08:06:47.248] [info] node id:22 process eof, add node to scheduler


Qavg: 850.216


[2023-07-11 08:06:50.954] [info] node id:22 Process node end


x265 [info]: frame I:      3, Avg QP:26.32  kb/s: 654.48  
x265 [info]: frame P:     61, Avg QP:24.68  kb/s: 5386.23 
x265 [info]: frame B:    236, Avg QP:29.70  kb/s: 302.40  
x265 [info]: Weighted P-Frames: Y:14.8% UV:13.1%
x265 [info]: consecutive B-frames: 4.7% 3.1% 1.6% 0.0% 90.6% 

encoded 300 frames in 69.18s (4.34 fps), 1339.63 kb/s, Avg QP:28.64
[aac @ 0x7f362812dfc0] Qavg: 850.216


[2023-07-11 08:06:50.979] [info] node id:22 close node
[2023-07-11 08:06:50.979] [info] node 22 close report, closed count: 4
[2023-07-11 08:06:50.979] [info] schedule queue 0 start to join thread
[2023-07-11 08:06:50.979] [info] schedule queue 0 thread quit
[2023-07-11 08:06:50.980] [info] schedule queue 0 closed
[2023-07-11 08:06:50.980] [info] schedule queue 1 start to join thread
[2023-07-11 08:06:50.980] [info] schedule queue 1 thread quit
[2023-07-11 08:06:50.980] [info] schedule queue 1 closed
[2023-07-11 08:06:50.980] [info] all scheduling threads were joint


In [23]:
! ffprobe decode_encode_multi_output0.mp4
! echo "----------------------------------------------------------------------"
! ffprobe decode_encode_multi_output1.mp4

! rm -rf decode_encode_multi_output0.mp4 decode_encode_multi_output1.mp4

ffprobe version 4.2.7-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --

### 2.2 audio transcoding


This demo shows how to use BMF to decode the input video, extract the audio part, add a piece of null audio before and after, and then encode the output wav file:

In [24]:
import bmf

input_video_path = "./big_bunny_10s_30fps.mp4"
output_path = "./with_null_audio.wav"

# create graph
graph = bmf.graph()

# decode
streams = graph.decode({
    "input_path": input_video_path
})

# create a null audio stream
audio_stream1 = graph.anullsrc('r=48000', 'cl=2').atrim('start=0', 'end=6')
audio_stream2 = graph.anullsrc('r=48000', 'cl=2').atrim('start=0', 'end=6')
concat_audio = (
    bmf.concat(audio_stream1, streams['audio'], audio_stream2, n=3, v=0, a=1)
)

(
    bmf.encode(
        None,
        concat_audio,
        {
            "output_path": output_path,
            "audio_params": {
                "codec": "aac",
                "bit_rate": 128000,
                "sample_rate": 44100,
                "channels": 2
            }
        }
    )
    .run()
)

{
    "input_streams": [],
    "output_streams": [],
    "nodes": [
        {
            "module_info": {
                "name": "c_ffmpeg_decoder",
                "type": "",
                "path": "",
                "entry": ""
            },
            "meta_info": {
                "premodule_id": -1,
                "callback_binding": []
            },
            "option": {
                "input_path": "./big_bunny_10s_30fps.mp4"
            },
            "input_streams": [],
            "output_streams": [
                {
                    "identifier": "audio:c_ffmpeg_decoder_23_1",
                    "stream_alias": ""
                }
            ],
            "input_manager": "immediate",
            "scheduler": 0,
            "alias": "",
            "id": 23
        },
        {
            "module_info": {
                "name": "c_ffmpeg_filter",
                "type": "",
                "path": "",
                "entry": ""
            },
        

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './big_bunny_10s_30fps.mp4':


[2023-07-11 08:06:51.375] [info] c++ module constructed
[2023-07-11 08:06:51.375] [info] Constructing c++ module
[2023-07-11 08:06:51.375] [info] c++ module constructed
[2023-07-11 08:06:51.375] [info] Constructing c++ module
[2023-07-11 08:06:51.375] [info] c++ module constructed
[2023-07-11 08:06:51.375] [info] Constructing c++ module
[2023-07-11 08:06:51.375] [info] c++ module constructed
[2023-07-11 08:06:51.375] [info] Constructing c++ module
[2023-07-11 08:06:51.375] [info] c++ module constructed
[2023-07-11 08:06:51.375] [info] Constructing c++ module
[2023-07-11 08:06:51.375] [info] c++ module constructed
[2023-07-11 08:06:51.375] [info] Constructing c++ module
[2023-07-11 08:06:51.375] [info] c++ module constructed


  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.76.100
  Duration: 00:00:10.00, start: 0.000000, bitrate: 2044 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 1920x1080 [SAR 1:1 DAR 16:9], 1904 kb/s, 30 fps, 30 tbr, 15360 tbn, 60 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 129 kb/s (default)
    Metadata:
      handler_name    : SoundHandler


[2023-07-11 08:06:51.382] [info] BMF Version: 0.0.5
[2023-07-11 08:06:51.385] [info] BMF Commit: 98e17ef
[2023-07-11 08:06:51.385] [info] start init graph
[2023-07-11 08:06:51.385] [info] scheduler count2
debug queue size, node 23, queue size: 5
[2023-07-11 08:06:51.385] [info] node:c_ffmpeg_decoder 23 scheduler 0
debug queue size, node 29, queue size: 5
[2023-07-11 08:06:51.386] [info] node:c_ffmpeg_encoder 29 scheduler 1
debug queue size, node 24, queue size: 5
[2023-07-11 08:06:51.386] [info] Constructing c++ module
[2023-07-11 08:06:51.386] [info] c++ module constructed
[2023-07-11 08:06:51.386] [info] node:c_ffmpeg_filter 24 scheduler 0
[2023-07-11 08:06:51.388] [info] node id:29 eof received
[2023-07-11 08:06:51.389] [info] push eof to orphan stream c_ffmpeg_encoder_29_1
[2023-07-11 08:06:51.402] [info] node id:29 eof processed, remove node from scheduler


Output #0, mp4, to './with_null_audio.wav':
  Metadata:
    encoder         : Lavf58.29.100
    Stream #0:0: Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 128 kb/s


[2023-07-11 08:06:51.667] [info] node id:23 decode flushing
[2023-07-11 08:06:51.677] [info] node id:23 Process node end
[2023-07-11 08:06:51.689] [info] node id:23 close node
[2023-07-11 08:06:51.691] [info] node 23 close report, closed count: 1
[2023-07-11 08:06:51.691] [info] node id:24 eof received
[2023-07-11 08:06:51.691] [info] node id:24 eof processed, remove node from scheduler
[2023-07-11 08:06:51.692] [info] node id:24 process eof, add node to scheduler
[2023-07-11 08:06:51.703] [info] node id:24 Process node end
[2023-07-11 08:06:51.705] [info] node id:24 close node
[2023-07-11 08:06:51.705] [info] node 24 close report, closed count: 2
[2023-07-11 08:06:51.705] [info] node id:29 eof received
[2023-07-11 08:06:51.705] [info] node id:29 eof processed, remove node from scheduler
[2023-07-11 08:06:51.705] [info] node id:29 process eof, add node to scheduler
[2023-07-11 08:06:51.757] [info] node id:29 Process node end
[2023-07-11 08:06:51.758] [info] node id:29 close node
[2023-

[aac @ 0x7f3600163d00] Qavg: 36541.152


[2023-07-11 08:06:51.758] [info] schedule queue 0 closed
[2023-07-11 08:06:51.758] [info] schedule queue 1 start to join thread
[2023-07-11 08:06:51.758] [info] schedule queue 1 thread quit
[2023-07-11 08:06:51.758] [info] schedule queue 1 closed
[2023-07-11 08:06:51.758] [info] all scheduling threads were joint


In [25]:
! ffprobe with_null_audio.wav
! rm -rf with_null_audio.wav

ffprobe version 4.2.7-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --

### 2.3 image transcoding
This demo shows how to decode a video, detect and skip black frames, encode and output to the pipeline, and then write files at the application layer. You can also do other custom operations, such as uploading directly to the cloud to avoid disk IO:

In [26]:
import bmf

input_video_path = "./big_bunny_10s_30fps.mp4"

graph = bmf.graph()
streams = graph.decode({
    "input_path": input_video_path,
})
video = streams['video'].ff_filter("blackframe", threshold=32).ff_filter("metadata", "select:key=lavfi.blackframe.pblack:value=96:function=less")
vframes_num = 2
result = (
    bmf.encode(
        video,
        None,
        {
            "push_output": 1,
            "vframes": vframes_num,
            "format": "image2pipe",
            "avio_buffer_size": 65536, #16*4096
            "video_params": {
                "codec": "jpg",
                "width": 640,
                "height": 480
            },
        }
    )
    .start()
)
write_num = 0
for i, packet in enumerate(result):
    avpacket = packet.get(bmf.BMFAVPacket)
    data = avpacket.data.numpy()
    if write_num < vframes_num:
        output_path = "./simple_image" + str(write_num)+ ".jpg"
        write_num = write_num + 1
        with open(output_path, "wb") as f:
            f.write(data)

[2023-07-11 08:08:16.110] [info] Constructing c++ module
[2023-07-11 08:08:16.127] [info] c++ module constructed
[2023-07-11 08:08:16.127] [info] Constructing c++ module
[2023-07-11 08:08:16.127] [info] c++ module constructed
[2023-07-11 08:08:16.127] [info] Constructing c++ module
[2023-07-11 08:08:16.127] [info] c++ module constructed
[2023-07-11 08:08:16.127] [info] Constructing c++ module
[2023-07-11 08:08:16.127] [error] node id:33 No output path
[2023-07-11 08:08:16.127] [info] c++ module constructed
[2023-07-11 08:08:16.127] [info] BMF Version: 0.0.5
[2023-07-11 08:08:16.127] [info] BMF Commit: 98e17ef
[2023-07-11 08:08:16.127] [info] start init graph
[2023-07-11 08:08:16.127] [info] scheduler count2
debug queue size, node 30, queue size: 5
[2023-07-11 08:08:16.127] [info] node:c_ffmpeg_decoder 30 scheduler 0
debug queue size, node 33, queue size: 5
[2023-07-11 08:08:16.127] [info] node:c_ffmpeg_encoder 33 scheduler 1
debug queue size, node 31, queue size: 5
[2023-07-11 08:08:16

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './big_bunny_10s_30fps.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.76.100
  Duration: 00:00:10.00, start: 0.000000, bitrate: 2044 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 1920x1080 [SAR 1:1 DAR 16:9], 1904 kb/s, 30 fps, 30 tbr, 15360 tbn, 60 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 129 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
[Parsed_blackframe_0 @ 0x7f3604af2f40] frame:0 pblack:100 pts:0 t:0.000000 type:? last_keyframe:0
[Parsed_blackframe_0 @ 0x7f3604af2f40] frame:1 pblack:100 pts:512 t:0.033333 type:? last_keyframe:0
[Parsed_blackframe_0 @ 0x7f3604af2f40] frame:2 pblack:100 pts:1024 t:0.066667 type:? last_keyframe:0
[Parsed_blackframe_0 @ 0x7f3604af2f40] frame:3 pblack:100 pts:1536 t:0

[2023-07-11 08:08:16.265] [info] *** 1 dup!
[2023-07-11 08:08:20.081] [info] node id:30 decode flushing
[2023-07-11 08:08:20.081] [info] node id:30 Process node end
[2023-07-11 08:08:20.091] [info] node id:30 close node
[2023-07-11 08:08:20.091] [info] node 30 close report, closed count: 1
[2023-07-11 08:08:20.091] [info] node id:31 eof received
[2023-07-11 08:08:20.091] [info] node id:31 eof processed, remove node from scheduler
[2023-07-11 08:08:20.101] [info] node id:31 process eof, add node to scheduler
[2023-07-11 08:08:20.127] [info] node id:31 Process node end
[2023-07-11 08:08:20.136] [info] node id:31 close node
[2023-07-11 08:08:20.139] [info] node 31 close report, closed count: 2
[2023-07-11 08:08:20.140] [info] node id:33 eof received
[2023-07-11 08:08:20.141] [info] node id:33 eof processed, remove node from scheduler
[2023-07-11 08:08:20.141] [info] node id:33 process eof, add node to scheduler
[2023-07-11 08:08:20.141] [info] *** dropping frame 2 at ts 2048
[2023-07-11 0

In [27]:
! ffprobe simple_image0.jpg
! rm -rf simple_image*.jpg

ffprobe version 4.2.7-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --